In [108]:
import math
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle
from prettytable import PrettyTable
%matplotlib inline
from PIL import Image
from sklearn.preprocessing import OneHotEncoder